In [1]:
# Import
import os
import pandas as pd
import cv2
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report,f1_score
import joblib


In [2]:
"""
read dataset according to the given file structure
weapon, label = 1
norm, label = 0
folder, the path to train folder only
"""
def MedianBlur(img, ksize):
    filteredImg = cv2.medianBlur(img, ksize=ksize)
    return filteredImg
def load_dataset(folder):
  data = []
  weap_path = folder + "/" + "weap"
  norm_path = folder + "/" + "norm"
  print(os.getcwd())
  for filename in os.listdir(weap_path):
    img = cv2.imread(weap_path + "/" + filename,0)
    img = MedianBlur(img,3)
    data.append([img,1])
    
  for filename in os.listdir(norm_path):
    img = cv2.imread(norm_path + "/" + filename,0)
    data.append([img,0])

  df = pd.DataFrame(data, columns=['image', 'label'])
  return df
data = load_dataset("frames/train")

/Users/zhangbokun/Documents/academic/4243/gunners


In [7]:
#resize image to (720, 1280)
X_data = []
y_data = []
for index, row in data.iterrows():
    image_data = row['image'] # Access the image data
    label = row['label']
    img = cv2.resize(src = image_data, dsize=(500,300)).flatten()
    X_data.append(img)
    y_data.append(label)
svm_classifier = SVC(kernel='rbf', C=0.1, random_state=42)
svm_classifier.fit(X_data, y_data)
joblib.dump(svm_classifier, 'svm_classifier.pkl')

['svm_classifier.pkl']

In [5]:
test_X = []
y_true = []
test = load_dataset("frames/test")
for index, row in test.iterrows():
    image_data = row['image'] # Access the image data
    label = row['label']
    test_X.append(cv2.resize(src = image_data, dsize=(500,300)).flatten().tolist() )
    y_true.append(label)
res=svm_classifier.predict(test_X)

print("accuracy: ", accuracy_score(test['label'].tolist(),res.tolist()))
print("f1: ",f1_score(y_true,res.tolist()))


/Users/zhangbokun/Documents/academic/4243/gunners
accuracy:  0.5519072911636891
f1:  0.2937595129375951


In [6]:
from sklearn.metrics import classification_report
print(classification_report(y_true, res, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.62      0.74      0.67      1292
           1       0.36      0.25      0.29       779

    accuracy                           0.55      2071
   macro avg       0.49      0.49      0.48      2071
weighted avg       0.52      0.55      0.53      2071

